# Movies recommandation

## 1. Imports

### 1.1 Libraries

In [23]:
# builtin
import os, time, sys, random

# data
import pandas as pd
import numpy as np
import requests
import math

# viz
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# NLP
import string
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.corpus import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
from collections import Counter

# ML
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# other
import warnings
warnings.filterwarnings("ignore")

### 1.2 Download and options

In [20]:
nltk.download('stopwords')
nltk.download('wordpunct')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading wordpunct: Package 'wordpunct' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\derou\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
sns.set()

### 1.3 Loading data

In [4]:
# CSV
df = pd.read_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\df_movies_cleaned.csv")

## 2. Work on a specific document

In [6]:
# Fonction de saut de ligne pour output
def insert_newlines(string, every=80):
    lines = []
    for i in range(0, len(string), every):
        lines.append(string[i:i+every])
    return '\n'.join(lines)

In [9]:
random_doc = df.Synopsis.sample(1)
random_doc = random_doc.values[0]
print(insert_newlines(random_doc, every=200))

Les indices pleuvent et la traque est lancée ! L'affaire du siècle est sur le point d'éclater au grand jour dans Basil, Détective privé. La célèbre souris détective vous emmène dans les rues pavées de
 Londres en 1897. Un enlèvement suspect est le point de départ de cette palpitante aventure musicale. Olivia, la fille d'un fabricant de jouets londonien, se rend à Baker Street pour demander à Basil 
de l'aider à retrouver son père. L'assistant de Basil, le docteur Dawson, et Toby le chien fidèle vont lui donner un coup de main, et un coup de truffe, pour chercher des indices dans leur merveilleux
 monde miniature. La piste les mènera finalement au professeur Ratigan, un criminel sans pitié que Basil va devoir duper pour sauver le royaume des souris !


### 2.1 Lower

In [10]:
random_doc = random_doc.lower()

In [15]:
print(insert_newlines(random_doc, every=200))

TypeError: sequence item 0: expected str instance, list found

### 2.2 Tokenization

In [19]:
tokens = word_tokenize(random_doc)
tokens

['les',
 'indices',
 'pleuvent',
 'et',
 'la',
 'traque',
 'est',
 'lancée',
 '!',
 "l'affaire",
 'du',
 'siècle',
 'est',
 'sur',
 'le',
 'point',
 "d'éclater",
 'au',
 'grand',
 'jour',
 'dans',
 'basil',
 ',',
 'détective',
 'privé',
 '.',
 'la',
 'célèbre',
 'souris',
 'détective',
 'vous',
 'emmène',
 'dans',
 'les',
 'rues',
 'pavées',
 'de',
 'londres',
 'en',
 '1897.',
 'un',
 'enlèvement',
 'suspect',
 'est',
 'le',
 'point',
 'de',
 'départ',
 'de',
 'cette',
 'palpitante',
 'aventure',
 'musicale',
 '.',
 'olivia',
 ',',
 'la',
 'fille',
 "d'un",
 'fabricant',
 'de',
 'jouets',
 'londonien',
 ',',
 'se',
 'rend',
 'à',
 'baker',
 'street',
 'pour',
 'demander',
 'à',
 'basil',
 'de',
 "l'aider",
 'à',
 'retrouver',
 'son',
 'père',
 '.',
 "l'assistant",
 'de',
 'basil',
 ',',
 'le',
 'docteur',
 'dawson',
 ',',
 'et',
 'toby',
 'le',
 'chien',
 'fidèle',
 'vont',
 'lui',
 'donner',
 'un',
 'coup',
 'de',
 'main',
 ',',
 'et',
 'un',
 'coup',
 'de',
 'truffe',
 ',',
 'pour',


In [17]:
# longueur de la liste
len(tokens)

142

In [18]:
# longueur de la liste (sans les doublons)
len(set(tokens))

92

In [27]:
def display_tokens_infos(tokens):
    """display info about corpus"""

    print(f"nb tokens {len(tokens)}, nb tokens uniques {len(set(tokens))}")
    print(tokens[:30])

In [30]:
tokens = wordpunct_tokenize(random_doc)
display_tokens_infos(tokens)

nb tokens 153, nb tokens uniques 94
['les', 'indices', 'pleuvent', 'et', 'la', 'traque', 'est', 'lancée', '!', 'l', "'", 'affaire', 'du', 'siècle', 'est', 'sur', 'le', 'point', 'd', "'", 'éclater', 'au', 'grand', 'jour', 'dans', 'basil', ',', 'détective', 'privé', '.']


### 2.3 Stopwords

In [32]:
stop_words = set(stopwords.words('french'))

In [36]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 95, nb tokens uniques 71
['indices', 'pleuvent', 'traque', 'lancée', '!', "'", 'affaire', 'siècle', 'point', "'", 'éclater', 'grand', 'jour', 'basil', ',', 'détective', 'privé', '.', 'célèbre', 'souris', 'détective', 'emmène', 'rues', 'pavées', 'londres', '1897', '.', 'enlèvement', 'suspect', 'point']


In [37]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(random_doc)
display_tokens_infos(tokens)

nb tokens 133, nb tokens uniques 90
['les', 'indices', 'pleuvent', 'et', 'la', 'traque', 'est', 'lancée', 'l', 'affaire', 'du', 'siècle', 'est', 'sur', 'le', 'point', 'd', 'éclater', 'au', 'grand', 'jour', 'dans', 'basil', 'détective', 'privé', 'la', 'célèbre', 'souris', 'détective', 'vous']


In [38]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 75, nb tokens uniques 67
['indices', 'pleuvent', 'traque', 'lancée', 'affaire', 'siècle', 'point', 'éclater', 'grand', 'jour', 'basil', 'détective', 'privé', 'célèbre', 'souris', 'détective', 'emmène', 'rues', 'pavées', 'londres', '1897', 'enlèvement', 'suspect', 'point', 'départ', 'cette', 'palpitante', 'aventure', 'musicale', 'olivia']


## 2. Work on the entire corpus

XXXXXXXXX

In [ ]:
df['Synopsis'] = df['Synopsis'].str.lower() 

In [ ]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
df_train['Synopsis'] = df_train['Synopsis'].apply(tokenizer.tokenize)

In [ ]:
#fonction pour supprimer les pronoms, articles, determinants etc 


stop_words = set(stopwords.words('french'))

def remove_stop_words(token_list):
    return [word for word in token_list if word not in stop_words]

In [ ]:
# Application de la fonction à la colonne Synopsis

df_train['Synopsis'] = df_train['Synopsis'].apply(remove_stop_words)

In [ ]:
#Création d'une liste de la colonne Synopsis
all_words = [word for token_list in df_train['Synopsis'] for word in token_list]

In [ ]:
filtered_frequencies = Counter(all_words)

# Count the frequencies of the remaining words
filtered_word_frequencies = Counter(filtered_frequencies)

most_common_words = filtered_word_frequencies.most_common(30) 

# Display the 10 most common words
print(most_common_words)

In [ ]:
#wordcloud

In [ ]:
#affiner la liste de stop words à partir des mots les + fréquents ? Genre "a", "plus", "où"....

In [ ]:
#stemming 

stemmer = FrenchStemmer()

def stem_words(token_list):
    return [stemmer.stem(word) for word in token_list]

df_train['Synopsis'] = df_train['Synopsis'].apply(stem_words)

In [ ]:
#Bag of words
model = Word2Vec(df_train['Synopsis'], vector_size=300, window=5, min_count=3, sg=1)

In [ ]:
model.train(df_train['Synopsis'], total_examples=len(df_train), epochs=10)

In [ ]:
similar_words = model.wv.most_similar('zomb')
similar_words

In [ ]:
# Vectorisation binaires des genres
print(len(df_train))
print(df_train['Genre'].apply(lambda x: isinstance(x, str)).sum())

In [ ]:
df_train['Genre'] = df_train['Genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)
print(df_train['Genre'].apply(lambda x: isinstance(x, list)).sum())

In [ ]:
mlb = MultiLabelBinarizer()
genre_binarized = mlb.fit_transform(df_train['Genre'])

# Créer un DataFrame avec les résultats
genre_df = pd.DataFrame(genre_binarized, columns=mlb.classes_)

In [ ]:
genre_df.index = df_train.index

In [ ]:
df = pd.concat([df_train.drop('Genre', axis=1), genre_df], axis=1)
df

In [ ]:
### Création du modèle de recommandation
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Synopsis'])

In [ ]:
# Calcul de la similarité cosinus
#cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
df['Synopsis'] = df['Synopsis'].apply(lambda x: ' '.join(x))
count_matrix = CountVectorizer().fit_transform(df['Synopsis'])

In [ ]:
indices = pd.Series(df.index, index=df['Titre']).to_dict()

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
# Fonction pour obtenir des recommandations
def get_recommendations(title, cosine_sim, df_movies_md, num_of_recs=10):

    title = title.lower()
    if title not in df_movies_md['Titre'].str.lower().values:
        return f"Aucune recommandation trouvée pour: {title}"
    
    # Obtenir l'indice du film donné son titre
    idx = df_movies_md[df_movies_md['Titre'].str.lower() == title.lower()].index[0]

    # Obtenir les scores de similarité pour ce film avec tous les films
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Multiplier par la note du film pour chaque score
    sim_scores = [(i, score * (df_movies_md.iloc[i]['Note'] / 10)) for i, score in sim_scores]

    # Trier les films en fonction des scores calculés
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenir les scores des n films les plus similaires
    sim_scores = sim_scores[1:num_of_recs+1]

    # Obtenir les indices de ces films
    movie_indices = [i[0] for i in sim_scores]

    # Retourner les films correspondants
    return df_movies_md.iloc[movie_indices]

In [ ]:
# Obtenir des recommandations pour un film donné
recommendations = get_recommendations("Coco", cosine_sim, indices)

# Afficher les recommandations
print(recommendations)
df[df['Titre'].str.contains("irréversible", case=False, na=False)]